In [39]:
import pandas as pd
import numpy as np

from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import plot_tree
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder


##Seaborn for fancy plots. 
import matplotlib.pyplot as plt
import seaborn as sns

import ml_utils
plt.rcParams["figure.figsize"] = (25,20)

## 3950 Assignment 1: Part 2

For this assignment we want to use some sort of tree based model to classify the data below. We have a very small training set, so overfitting is a very real concern. 

Some specifics for this assignment:
<ul>
<li>Please use the show_eda to control if EDA stuff is shown. I don't really need to see all the EDA stuff (nor do you after you've done it), so we can make it configurable with a variable to speed up time. Please set this FALSE when you submit, so I can run all and see the outcome without histograms etc...
<li>Please ensure that whatever model you end up with is in a variable named best at the end.
<li>Please use some pipeline in prepping the data. The test data is in an identical format to the training data, so whatever pipeline you've created for your training will work for the testing. 
<li>The accuracy scoring will be an average of accuracy and roc_auc. 
</ul>

### Grading Metrics
<ul>
<li><b>Pipeline Used - 10pts</b> The data loading needs to be in a pipeline. See the test part for illustration. When testing I'll call your pipe with the new data (format is identical to training), so any prep stuff should be in the pipeline. 
<li><b>Tree Based Model Used - 5pts</b> The model used for classification needs to be some variety of tree, beyond that it is up to you. 
<li><b>Accuracy - 5pts</b> The final accuracy acheived. This will be a rough ranking, I'm assuming most people will get a similar level of accuracy, marks will only be deducted if yours is far wosrse, as that's an indication that you probably didn't take any/many steps to improve things. 
<li><b>Clarity and Formatting - 5pts</b> Is it organized and can I read it?
    <ul>
    <li> <b>Note:</b> for this assignment, and in general, please get rid of my comments and replace them with your own. I'm going to read this, so all of these instructions aren't really required. Think of this as a template, get rid of the stuff that isn't needed, and leave only the things you need to explain your code. 
    </ul>
</ul>

For submission, please drop the URL for your repository in the dropbox.

In [6]:
#Please change to your name.
name = "Adam Sykes"

#Please use this to control EDA. 
show_eda = False

In [3]:
#Load data
df = pd.read_csv("training.csv")
df = df.drop(columns={"id"})
df.sample(5)

,target,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199,var_200
209,1,0.201,0.499,0.702,0.746,0.839,0.316,0.303,0.025,0.996,...,0.096,0.292,0.771,0.566,0.167,0.347,0.049,0.652,0.402,0.413
145,0,0.334,0.946,0.829,0.959,0.420,0.917,0.469,0.622,0.924,...,0.059,0.495,0.880,0.084,0.083,0.192,0.732,0.948,0.482,0.268
7,1,0.984,0.907,0.207,0.042,0.518,0.585,0.585,0.913,0.131,...,0.680,0.429,0.691,0.813,0.405,0.695,0.821,0.889,0.030,0.513
173,0,0.443,0.999,0.527,0.291,0.140,0.608,0.848,0.132,0.454,...,0.882,0.236,0.031,0.512,0.155,0.904,0.813,0.054,0.117,0.509
36,0,0.403,0.840,0.344,0.322,0.102,0.059,0.982,0.768,0.937,...,0.083,0.558,0.782,0.603,0.346,0.436,0.397,0.265,0.096,0.234


### Starting

For this assignment, you have a small training set, so combatting overfitting is key in being accurate!

In [4]:
dfEDA = ml_utils.edaDF(df,'target')

In [7]:
if show_eda == True:
    dfEDA.fullEDA()

#### Do Modelling Stuff

Make a tree model (of some vareity) and make it fit well. Keep in mind the possibility of your tree overfitting, and think of steps you may need to combat that shoudl it occur. 

In [53]:
y = df["target"]
x = df.drop(columns={"target"})

xTrain, xTest, yTrain, yTest = train_test_split(x, y)

In [52]:
rf_para = {
           'min_samples_split': range(5,30,5),
           'min_samples_leaf': range(10,51,5),
           'max_leaf_nodes': range(3,25,3)
}
 
clf = GridSearchCV(estimator=DecisionTreeClassifier(random_state=0), param_grid=rf_para, n_jobs=-1) 
clf.fit(xTrain, yTrain.ravel())
print(clf.best_estimator_)
print("Training Accuracy:", clf.best_estimator_.score(xTrain, yTrain))
print("Testing Accuracy:", clf.best_estimator_.score(xTest, yTest))

DecisionTreeClassifier(max_leaf_nodes=3, min_samples_leaf=40,
                       min_samples_split=5, random_state=0)
Training Accuracy: 0.7005347593582888
Testing Accuracy: 0.5555555555555556


In [57]:
imputer_transformer = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="mean"))
])

numeric_transformer = Pipeline(steps=[
        ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
        ('onehot', OneHotEncoder(drop='first'))
])

preprocessor = ColumnTransformer(transformers=[
        ('imp', imputer_transformer, dfEDA.num),
        ("num", numeric_transformer, dfEDA.num),
        ('cat', categorical_transformer, dfEDA.cat)
])

pipeline_steps = [('pre', preprocessor),('RF', DecisionTreeClassifier(max_leaf_nodes=3, min_samples_leaf=40,
                       min_samples_split=5, random_state=0))]

best = Pipeline(pipeline_steps)
best.fit(xTrain,yTrain)

Pipeline(steps=[('pre',
                 ColumnTransformer(transformers=[('imp',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  ['var_1', 'var_2', 'var_3',
                                                   'var_4', 'var_5', 'var_6',
                                                   'var_7', 'var_8', 'var_9',
                                                   'var_10', 'var_11', 'var_12',
                                                   'var_13', 'var_14', 'var_15',
                                                   'var_16', 'var_17', 'var_18',
                                                   'var_19', 'var_20', 'var_21',
                                                   'var_22', 'var_23', 'var_24',
                                                   'var_25', 'var_26', 'var_27',
                                                   'var_28...
                                                   'var_10', 'var_11', 'var_12',
                                                   'var_13', 'var_14', 'var_15',
                                                   'var_16', 'var_17', 'var_18',
                                                   'var_19', 'var_20', 'var_21',
                                                   'var_22', 'var_23', 'var_24',
                                                   'var_25', 'var_26', 'var_27',
                                                   'var_28', 'var_29', 'var_30', ...]),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(drop='first'))]),
                                                  [])])),
                ('RF',
                 DecisionTreeClassifier(max_leaf_nodes=3, min_samples_leaf=40,
                                        min_samples_split=5, random_state=0))])

### Finishing

At the conclusion, please name your best model "best". If you look down below in the testing stuff, it should be usable to score as "best". 

You should be able to call it like this and it should work (with whatever data names you have)

In [58]:
print(best.score(xTest, yTest))
print(best._final_estimator)

0.49206349206349204
DecisionTreeClassifier(max_leaf_nodes=3, min_samples_leaf=40,
                       min_samples_split=5, random_state=0)


### Testing

Please leave the stuff below as-is in your file. 

This will take your best model and score it with the test data. If you want to test to make sure that yours works, make a copy of the data file and rename it testing.csv, then make sure this runs ok. I will do the same, but the contents of my test file will be different. 

In [ ]:
#Load Test Data
test_df = pd.read_csv("testing.csv")
#test_df = test_df.drop(columns={"id"})
#Create tests and score
test_y = np.array(test_df["target"]).reshape(-1,1)
test_X = np.array(test_df.drop(columns={"target"}))

preds = best.predict(test_X)

roc_score = roc_auc_score(test_y, preds)
acc_score = accuracy_score(test_y, preds)

print(roc_score)
print(acc_score)
print(name, np.mean([roc_score, acc_score]))


### What Accuracy Changes Were Used

Please list here what you did to try to increase accuracy and/or limit overfitting:
<ul>
<li>Maximum number of leaf nodes allowed
<li>Minimum samples a node must have to enable a split
<li>Minimum samples a leaf node must contain
</ul>